### Importo TODO indiscriminadamente 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import seaborn as sns
import scipy.stats as stats
from scipy.optimize import curve_fit

### Codigo

#### Importar una matriz de datos y ponerla en un array lineal

In [ ]:
path = ''
datos = np.genfromtxt(path + '',delimiter= ';').reshape(1,300)

# Hay que cambiar 300 con el largo del nuevo array

#### Funcion para sacar los intervalos de confianza

In [ ]:
T = []

def intervalos(data):
  x0 = np.round(np.mean(data), decimals = 3)
  s = np.round(np.std(data), decimals = 3)
  print(f'Valor medio: {x0}, Desv. Estandar: {s}')
  print(f'Intervalo 68% = ({np.round(x0-s, decimals = 3)} ; {np.round(x0+s, decimals = 3)})s') 
  print(f'Intervalo 95% = ({np.round(x0-2*s, decimals = 3)} ; {np.round(x0+2*s, decimals = 3)})s') 
  print(f'Intervalo 99% = ({np.round(x0-3*s, decimals = 3)} ; {np.round(x0+3*s, decimals = 3)})s') 

intervalos(T)

#### Funcion para leer archivos del photogate

In [4]:
def leer_data(data, largo):
    def leermd(filename):
        result = []
        df = read_csv(filename, sep='\t', skiprows=3, decimal=',')
        nparray = df.values
        nparray = nparray[~np.isnan(nparray).any(axis=1)]
        for cols in range(nparray.shape[1]):
            out = nparray[:,cols]
            result.append(out)
        return result      
    #Lo de arriba no lo hice yo :P

    # VAS A TENER QUE CAMBIAR EL PATH
    path = ''
    
    ts, ph = leermd(path+data)

    dph = np.diff(ph)

    idx_i = np.where(dph<-2)[0]
    idx_d = np.where(dph>2)[0]-1

    tiempo_d = ts[idx_d]
    tiempo_i = ts[idx_i]

    punto_medio = tiempo_i + (tiempo_d - tiempo_i)/2 

    datos = []
    for i in range(len(punto_medio)):
        if i % 2 != 0:
            datos.append(punto_medio[i])

    periodos = np.diff(datos)
    
    x0 = np.round(np.mean(periodos), decimals = 3)
    s = np.round(np.std(periodos), decimals = 3)
    print(f'Cantidad de periodos: {len(periodos)}')
    print(f'Valor medio: {x0} \ Desviacion estandar: {s}')
    
    g = (4*(np.pi**2)*largo)/x0**2
    g = np.round(g, decimals=3)
    print(f'El valor de g con el valor medio es: {g}')

    return periodos 

#### Empiezan las cosas de Histogramas

Mezclar y dividir datos

In [ ]:
# Pone el path
path = ''
datos = np.genfromtxt(path)
np.random.seed(0)
np.random.shuffle(datos)
d = np.split(datos, 10)

Defino una funcion bastante general para graficar histogramas

In [12]:
def todo(i, b, k):

    # b define el ancho de los bins 
    # Defino el Valor medio(x) y la Desviacion Media(s)
    x = np.round(np.mean(i), decimals=2)
    s = np.round(np.std(i), decimals=2)   
    print(f'Valor Medio: {x} - Desviacion Estandar: {s}')

    # Dibujo el Histograma de cada Grupo
    sns.histplot(i,
                 binwidth= b, 
                 binrange=(1.50,2.10),
                 color='lightblue', edgecolor='black'
                )
    
    plt.title(f'Grupo {k}', fontsize=25)
    plt.grid(axis='y', linestyle='-.', color='black')
    plt.xlabel('Tiempo [s]', fontsize=15)
    plt.ylabel('Cantidad de Repeticiones', fontsize=15)
    plt.xlim(1.45,2.15)
    plt.show()
    return

#• Va a haber que cambiar el 'binrange' y el 'plt.xlim'
#• stat = 'probability' normaliza el histograma como quieren aca


#### Defino una funcion para graficar las gaussianas asociadas a cada Grupo (sin normalizar)

In [19]:
def gauss(i,j,k,l):

    """
    Para llamar la funcion debo dar las siguientes variavles:
    i = valor medio
    j = Desviacion estandar
    k = color de cada funcion
    l = Etiqueta de cada funcion
    """
    # Creo un array con mil datos entre los valores que necesite (los del binrange)
    X = np.linspace( , ,1000)
    Y = stats.norm.pdf(X, loc= i , scale= j)
    plt.plot(X,Y , color = k, label= l)
    plt.legend()

#### Grafico la curva ajustada al histograma de todos los datos

In [ ]:
# Histograma

#• Va a haber que cambiar el 'binrange', el 'plt.xlim' y el 'binwidth'
sns.histplot(datos,
              stat = 'probability',
              binwidth= 0.10, 
              binrange=(1.50,2.10),
              alpha= 0.5,
              color='lightblue' 
            )

# Puntos de la Tablita
X = [] # el valor medio de cada columna
Y = [] # suerte

plt.scatter(X,Y, color= 'black')


# Curva
def f_g(i,u,d,a):
  Y = (a)/(d*np.sqrt(2*np.pi))*np.exp(-(i-u)**2/(2*(d**2)))
  return Y

popt,pcov = curve_fit(f_g,X,Y,p0=[1.8,0.11,0.09])
x_ = popt[0]
s_ = popt[1]
a_ = popt[2]

eje_x = np.linspace(1.45,2.15,1000)
eje_y = f_g(eje_x,x_,s_,a_)

plt.plot(eje_x ,eje_y, color= 'red')

plt.xlabel('Tiempo [s]', fontsize=15)
plt.ylabel('Conteo', fontsize=15)
plt.xlim(1.45,2.15)
plt.grid()
plt.show()

Esta es la linea de codigo que use para sacar las alturas de las columnas

In [ ]:
import pandas as pd

serie = pd.Series(datos)
hist = serie.hist(bins=6, range=(1.50,2.10))

def get_hist(ax):
    n,bins = [],[]
    for rect in ax.patches:
        ((x0, y0), (x1, y1)) = rect.get_bbox().get_points()
        n.append(y1-y0)
        bins.append(x0) # left edge of each bin
    bins.append(x1) # also get right edge of last bin

    return n,bins

n, bins = get_hist(hist)

print(n)

lol=[]
for i in range(0,6):
  lol.append(np.round(n[i] / 600, decimals=3))

print(lol)
plt.show()

#• Obviamente voy a tener que cambiar los bins y el range en el histograma 
#• Tambien va a haber que cambiar el '600' por la cantidad de datos que tenga la lista